In [ ]:
from dotenv import load_dotenv
import os 
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel


ollama_llm = Ollama(model = 'llama3')

load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')
Model = 'gpt-3.5-turbo'
gpt_llm = ChatOpenAI(api_key = API_KEY,model= Model)

gpt_llm.invoke('what is a bot')

parser = StrOutputParser()
gpt_chain = gpt_llm|parser
gpt_chain.invoke('what is a bot')

loader = TextLoader('data.txt',encoding = 'utf-8')
document = loader.load()

spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
chunks = spliter.split_documents(document)
chunks[3]

vector_storage = FAISS.from_documents(chunks,OpenAIEmbeddings())
retriever = vector_storage.as_retriever()

retriever.invoke('what is the pricing for Bhuman')

template = ("""
You are AI-powered chatbot designed to provide 
information and assistance for customers
based on the context provided to you only. 
            
Context:{context}
Question:{question}
""")

prompt = PromptTemplate.from_template(template=template)
prompt.format(
    context = ' Here is a context to use',
    question = ' This is a question to answer'
)

result = RunnableParallel(context = retriever,question = RunnablePassthrough())
chain = result |prompt | gpt_llm | parser

chain.invoke('What is the pricing plan for Bhuman')